In [ ]:
include("mdlecturesrc.jl")

# Lecture 2: Molecular Dynamics

* Newton's Law from the Schrodinger equation
* Integration, Discretisation
* Langevin dynamics
* Hydrogen bond in the water dimer
* Statistical errors on observables
* Thermal expansion of the water hexamer

## Modeling the world

As far as we know, the world works according to the Schrödinger equation:

$$
-i\hbar \frac{\partial \Psi}{\partial t} = \mathcal{H} \Psi
$$

where $\Psi$ is a _wave function_ (or probability amplitude),

$$
\Psi \equiv \Psi(x_1, x_2, x_3, \ldots)\qquad x_i \in \mathrm{R}^3
$$

Take the particles to be electrons and atomic nuclei, then the Hamiltonian $\mathcal{H}$ is

$$
\mathcal{H} = \sum_i \frac{1}{2m_i} \nabla^2_{x_i} + \sum_{ij} \frac{Z_i Z_j}{\left|x_i-x_j\right|}
$$

* The first term gives matter is "wavelike" character, the particles' tendency to spread is inversely proportional to their mass
* The second term is the Coulomb interaction of charges. It is easily observable macroscopically!
* (There is also spin, which is omitted from above for simplicity... wave function is a really a 4-vector ... see Dirac theory)
* Limitation: particles stay the same (below some energy limit, they do!)
* Thus far not contradicted by any experiment

In addition, for Fermions (e.g. electrons), the wave function is _antisymmetric_,

$$
\Psi(\ldots, x_i, \ldots, x_j, \ldots) = - \Psi(\ldots, x_j, \ldots, x_i, \ldots) 
$$


## Hierarchy of approximations

Formal solution of the wave equation is

$$
\Psi(t) = e^{i \mathcal{H} t /\hbar} \Psi_0
$$

So eigenvectors of $\mathcal{H}$ are stationary states of an isolated system.
* For a system that interacts with the environment at temperature $T$, the probability of occupying an eigen(energy)-level with energy $E_i$ follows the Boltzmann distribution,
$$
\\
P(\Psi_i) \propto e^{-E_i/kT}
\\
$$
* Room temperature is _small_, $kT \approx 0.025~\mathrm{eV}$. Typical electronic transitions are on the order of $1~ \mathrm{eV}$, e.g. for visible light $\hbar c/\lambda \approx 2~\mathrm{eV}$

In [ ]:
using Physical

h_planck*c_light/(500*(Nano*Meter))/ElectronVolt

* Mass scale of electrons and protons are very different: $ 1~\mathrm{p} \approx 1836~\mathrm{e}$, so electrons are _fast_.

* The Born-Oppenheimer approximation separates the eletronic motion (coordinates: $r$) from nuclear motion (coordinates: $R$)


#### The electronic problem

Nuclei fixed at positions $R_j$, consider electronic wave function $\Psi_\mathrm{el}(r_1,r_2,\ldots)$ with Hamiltonian (units: $\hbar = m_\mathrm{e} = e = 1$)

$$
\mathcal{H}_\mathrm{el} = \frac{1}{2} \sum_i \nabla^2_{r_i} + \sum_{ij} \frac{(-1)Z_j}{\left|R_j-r_i\right|} + \sum_{i i'} \frac{1}{\left|r_i-r_{i'}\right|}
$$

Typically we are interested in the situation in which electrons are relaxed into their ground state, the lowest eigenstate whose eigenvalue we identify with electronic potential energy $V_\mathrm{el}(R_1,R_2,\ldots) \equiv E_0$ 

#### The nuclear problem

Here the wave function is of the nuclear coordinates, $\Psi(R_1,R_2,\ldots)$ with Hamiltonian

$$
\mathcal{H}_\mathrm{el} = \sum_j \frac{1}{2m_j} \nabla^2_{R_j} + \sum_{jj'} \frac{Z_j Z_{j'}}{\left|R_j-R_{j'}\right|} + V_\mathrm{el}(R_1,R_2,\ldots)
$$

* At ambient conditions, the nuclear wave functions do not spread very much, do not overlap, and a separable solution is rather accurate (except for low mass hydrogen...)

$$
\Psi(R_1,R_2,\ldots) \approx \Psi(R_1)\Psi(R_2)\Psi(R_3)\ldots
$$

* The quantum system is reduced to a classical system via the _correspondence principle_

$$
\begin{eqnarray}
p_j &=& \langle \Psi | \nabla_{R_j} | \Psi\rangle \\
q_j &=& \langle \Psi | R_j | \Psi\rangle \\
H &=&  \langle \Psi | \mathcal{H}| \Psi\rangle 
\end{eqnarray}
$$

So the classical Hamiltonian is

$$
H = \sum_j \frac{p_j^2}{2m_j} + \underbrace{\sum_{jj'} \frac{Z_j Z_{j'}}{\left|q_j-q_{j'}\right|} + V_\mathrm{el}(q_1,q_2,\ldots)}_{\textrm{"interatomic potential": } V(q_1,q_2,\ldots)}
$$

Leading to the equations of motion, i.e. Newton's law, 

$$
\begin{eqnarray}
\dot q_j &=& \partial H/\partial p_j \qquad & \left(= p_j / m_j\right)\\
\dot p_j &=& -\partial H/\partial q_j \qquad & \left(= -\partial V/\partial q_j = F_j = m_j \ddot q_j \right)\\
\end{eqnarray}
$$

## Discretise, Integrate

We want to numerically integrate the equations of motion forward in time from a given starting point. Truncating the Taylor expansion,

$$
\begin{eqnarray}
q(t+h) &=& q(t) + h \left.\frac{d q}{d t}\right|_t + O(h^2)\\
p(t+h) &=& p(t) + h \left.\frac{d p}{d t}\right|_t + O(h^2)\\
\end{eqnarray}
$$

Leads to Euler's method,

$$
\begin{eqnarray}
q(t+h) &=& q(t) +  h p(t)/m\\
p(t+h) &=& p(t) +  h F(q(t))\\
\end{eqnarray}
$$

In [ ]:
Euler = ( (q,p,F,h) -> ( q+h*p, p+h*F(q) )  ) ;

### Try it on the simple harmonic oscillator

The potential is $V(q) = q^2$, with exact solution (starting from $q(0) = 0$),

$$
\begin{eqnarray}
q(t) &=& p(0) \sin(t)\\
p(t) &=& p(0) \cos(t)\\
\end{eqnarray}
$$

In [ ]:
sho_V = (q-> q^2/2) # the potential function
sho_F = (q-> -q)    # the force function
p0 = 0.1
kT = 0.01

f = figure()
@manipulate for h in [0.01,0.1,0.5,1.0,2.0],
                show=Dict(:Trajectory => 1, :Phasespace => 2, :Energy => 3),
                integrator=Dict(:velocityVerletLangevin => velocityVerletLangevin,
                                :velocityVerlet => velocityVerlet,
                                :Verlet => Verlet,
                                :Euler => Euler
                                ),
                 γ in [0.01, 0.1, 0.5, 1] withfig(f) do 

        
        
        
        t = 0:h:1000; q = zeros(t); p = zeros(t); p[1] = p0; Fold = sho_F(q[1])

        for i=2:length(t)
            if integrator === Euler

                q[i],p[i] = Euler(q[i-1],p[i-1], sho_F, h)
              
                
    
                
                
                
                
            elseif integrator === Verlet
                if i==2
                    q[i],p[i] = Euler(q[i-1], p0, sho_F, h)
                else
                    q[i] = Verlet(q[i-1], q[i-2], sho_F, h)
                    p[i-1] = (q[i]-q[i-2])/(2*h)
                end
            elseif integrator === velocityVerlet
                q[i], p[i], Fold = velocityVerlet(q[i-1], p[i-1], Fold, sho_F, h)
            elseif integrator === velocityVerletLangevin
                q[i], p[i], Fold = velocityVerletLangevin(q[i-1], p[i-1], Fold, sho_F, h, kT, γ)
            end
        end
        
        r = 1:length(find(t .< 20))
        if show==1
            plot(t[r], q[r], "o-", markersize=2)
            plot(t[r], p0*sin(t[r]), "r.", markersize= (h <0.5 ? 1 : 5))
            xlabel("t"); ylabel("q")
        elseif show==2
            marker = "o-"
            rr = r
            if integrator === velocityVerletLangevin
                r = 1:length(find(t .< 100))
                marker = "."
            end
            plot(q[r], p[r], marker, markersize=2)
            plot(p0*sin(t[rr]), p0*cos(t[rr]), "r.", markersize= (h <0.5 ? 1 : 5) )
            axis("equal"); xlabel("q"); ylabel("p")
        elseif show==3
            Eaxis = kT/4:kT/4:12*kT
            bins,Ehist = hist(map(sho_V, q)+p.^2/2, Eaxis)
            plot(midpoints(bins), Ehist/length(q)/(kT/4), "-")
            plot(Eaxis, exp(-Eaxis/kT)/kT, "r-")
        end
    end
end

### Verlet integrator

Instead of going to higher order in the Taylor expansion, we make it more symmetric

$$
\begin{eqnarray}
q(t-h) &=& q(t) - h \dot q(t) + \frac{h^2}{2} \ddot q(t) + O(h^3)\\
q(t+h) &=& q(t) + h \dot q(t) + \frac{h^2}{2} \ddot q(t) - O(h^3)\\
\end{eqnarray}
$$

Adding the two leads to

$$
q(t+h) = 2 q(t) - q(t-h) + h^2 \ddot q(t) = 2 q(t) - q(t-h) + h^2 F(q(t))/m
$$

In [ ]:
Verlet = ( (q,qq,F,h) -> ( 2*q-qq+h^2*F(q) ) ) ;

With this integrator, the velocity is not computed directly, it has to be obtained by finite difference after the step. The following variant called velocity-Verlet avoids this, but needs the force from the previous time step passed in.

$$
\begin{eqnarray}
p(t+\frac{h}{2}) &=& p(t) + \frac{h}{2} F(q(t))\\
q(t+h) &=& q(t) + h p(t+\frac{h}{2})/m \\
p(t+h) &=& p(t+\frac{h}{2}) + \frac{h}{2} F(q(t+h))
\end{eqnarray}
$$

In [ ]:
velocityVerlet = (  (q,p,Fq,F,h) -> (ph2 = p+h/2*Fq;
                                     qh = q+h*ph2;
                                     Fq = F(qh);
                                    (qh, ph2+h/2*Fq, Fq) )  );

### Langevin dynamics

To model an external heat bath, we introduce Langevin Dynamics, which involves random perturbations on each particle at every time step, and a corresponding frictional force to balance it out. 

The equations of motion are:

$$
\begin{eqnarray}
p(t+\frac{h}{2}) &=& p(t) + \frac{h}{2} F(q(t))\\
q(t+\frac{h}{2}) &=& q(t) + \frac{h}{2} p(t+\frac{h}{2})/m\\
\hat p(t+\frac{h}{2}) &=& e^{-\gamma h} p(t+\frac{h}{2})+\sqrt{kT(1-e^{-2\gamma h})}R(t)\\
q(t+h) &=& q(t+\frac{h}{2}) + \frac{h}{2} \hat p(t+\frac{h}{2})/m \\
p(t+h) &=& \hat p(t+\frac{h}{2}) + \frac{h}{2} F(q(t+h))
\end{eqnarray}
$$

where $T$ is the temperature of the heat bath, and $\gamma$ is the strength of the coupling. The limit of $\gamma \rightarrow 0$ corresponds to the isolated system, and reduces to velocity-Verlet. 

In [ ]:
function velocityVerletLangevin(q, p, Fq, F, h, kT, gamma)
    ph2 = p + h/2*Fq
    qh2 = q + h/2*ph2
    pph2= exp(-gamma*h)*ph2 + sqrt(kT*(1-exp(-2*gamma*h)))*(randn(1)[1])
    qh  = qh2 + h/2*pph2
    Fqh = F(qh)
    ph  = pph2 + h/2*Fqh
    (qh, ph, Fqh)
end
;

## Dynamics of the water dimer

In [ ]:
imolecule_draw(make_h4o2(optim=true))

In [ ]:
function water_dimer_dynamics(;Temp::Float64=1.0, Nsteps=100, Nsave=0)
    h4o2 = make_h4o2(optim=true)

    q = vec(h4o2[:get_positions]()')

    
    h=0.5            # time step
    m = repeat(h4o2[:get_masses](), inner=[3])*quippy.units[:MASSCONVERT] #masses to conform to eV, A, fs units
    Fq = -tip3p.gradient(q)
    p = zeros(q); tmp1 = zeros(q); tmp2 = zeros(q); tmp3 = zeros(q); rOO = zeros(Nsteps)
    
    Nsave > 0 ? (traj = quippy.CInOutput("traj.xyz", quippy.OUTPUT)) : 0
    
    for i=1:Nsteps
        rOO[i] = norm(q[1:3]-q[10:12])
        
        velocityVerletLangevin!(q, p, Fq, q->(-tip3p.gradient(q)), m, h, Temp, 0.1, tmp1, tmp2, tmp3) 
        
        if Nsave> 0 && (i-1)%Nsave == 0
            h4o2[:set_positions](reshape(q, (3,6))') ; traj[:write](h4o2)
        end 
    end
    
    Nsave > 0 ?  traj[:close]() : 0
    
    return rOO
end

In [ ]:
 @time water_dimer_dynamics(Temp=50.0, Nsteps=1000, Nsave=10);

In [ ]:
Temps = [1.0,10.0,50.0,100.0,300.0]
rOO = zeros(100000,length(Temps))
i=1
for T in Temps
    rOO[:,i] = water_dimer_dynamics(Temp=T, Nsteps=size(rOO,1), Nsave=0);
    println("T=$(Temps[i]) done")
    i += 1
end
println("done!")

In [ ]:
figure()
for i=length(Temps):-1:1
    plot(rOO[1:1:10000,i], "-", label="T=$(Temps[i])")
end
xlabel("iterations"); ylabel("rOO"); legend(loc="center left", bbox_to_anchor=(1, 0.5))
close(gcf())

In [ ]:
figure()
for i=length(Temps):-1:2
    plt[:hist](rOO[1:10:end,i], 20, label="T=$(Temps[i])", linewidth=0.2)
end
xlabel("rOO"); ylabel("count(rOO)");legend()
close(gcf())

In [ ]:
figure(figsize=(8,6))

bins = 2.4:0.02:4.5
for i=length(Temps):-1:1
    dummy,p = hist(rOO[:,i], bins)
    F = -log(p)*kB*Temps[i]

    plot(midpoints(bins), F - minimum(F), "o-", label="T=$(Temps[i])")
end

r,E = water_dimer_dissoc()
plot(r, E-minimum(E), "k", label="T=0")

axis([minimum(bins), 6.0, -0.01, 0.3]); legend(loc="center left", bbox_to_anchor=(1, 0.5)); xlabel(L"r_\mathrm{OO}"); ylabel("Energy")
close(gcf())

## Dynamics of the water hexamer

In [ ]:
h12o6 = make_h12o6()
imolecule_draw(h12o6)

In [ ]:
# takes about 2-3 minutes to execute

h=0.5
m = repeat(h12o6[:get_masses](), inner=[3])*quippy.units[:MASSCONVERT] #masses to conform to eV, A, fs units
q = vec(h12o6[:get_positions]()')
Fq = -tip3p.gradient(q)
p = zeros(q) ; tmp1 = zeros(q); tmp2 = zeros(q); tmp3 = zeros(q)
traj = quippy.CInOutput("traj.xyz", quippy.OUTPUT)
N=100001
Ekin = zeros(N); Epot = zeros(N); Rg = zeros(N); w1z = zeros(N)

@time for i=1:N
    Ekin[i] = sum(p.^2./m/2)
    #Epot[i] = tip3p.potential(q)
    Rg[i] = h12o6_Rg(q)
    w1z[i] = q[6]-q[3]
    if i%1 == 1
        h12o6[:set_positions](reshape(q, (3,18))')
        traj[:write](h12o6)
    end 
    if i%10000 == 1
        println("iteration $(i-1)")
    end
    velocityVerletLangevin!(q, p, Fq, q->(-tip3p.gradient(q)), m, h, 400.0, 0.1, tmp1, tmp2, tmp3)   
end
traj[:close]()

## Observables

Kinetic "temperature":

$$
E_\text{kin} \approx 3N\frac{1}{2}k_B T
$$

In [ ]:
T = Ekin*2/(3*18*kB) # kinetic temperature
;

In [ ]:
figure()
plot(T[1:100], "-")
plot(1:10:100, T[1:10:100], "ro")
xlabel("iteration"); ylabel("Kinetic temperature / K")
close(gcf())

In [ ]:
std(Ekin*2/(3*18*kB)), 400/sqrt(3*18)

## How can we compute the error on the sample mean? 

(The following discussion draws on A. Sokal's lecture notes on "Monte Carlo Methods in Statistical Mechanics")

* Estimating the variance of the mean of independent samples is easy

$$
\begin{eqnarray}
\bar x &=& \frac1N \sum_i^N x_i\\
\text{Var}(\bar x) &=& \frac{1}{N^2} N \text{Var}(x)\\
&=& \frac{\text{Var}(x)}{N}\\
\text{Std}(\bar x) &=& \frac{\text{Std}(x)}{\sqrt{N}}
\end{eqnarray}
$$

* We could resample the sequence and only keep indpendent samples
 * How long do we have to wait for samples to become independent?
 * Are we wasting a lot of data?
 

### Autocorrelation

Given the _true_ mean, $\mu_x = \langle x\rangle$

$$
\begin{eqnarray}
C_{xx} (i) &=& \frac1N\sum_{j=1}^N (x(j)-\mu_x) (x(j+i)-\mu_x)\\
&=& \langle x(j)x(j+i)\rangle - \mu_x^2\\
\rho_{xx} (i) &=& C_{xx}(i)/C_{xx}(0)
\end{eqnarray}
$$

* $C_{xx}(0)$ is just the variance of $x$
* The autocorrelation tells us how correlated samples are at different times 
* As separation increases, we expect samples to be less and less correlated 
* Autocorrelation therefore decays with separation 
* (Note: strictly, zero correlation does not imply independence)
* Problem: computing the autocorrelation is difficult, because it requires knowledge of the true mean! 

In [ ]:

Ns=100
mT = 400.0 # mean
C_T = fftshift(xcorr(T[1:Ns] - mT, T[1:Ns] - mT))[1:100]

rho_T = C_T/C_T[1]

figure()
plot( rho_T, "-")
close(gcf())


In [ ]:
figure()

for Ns in [100, 1000, 10000, 100000]
    mT = mean(T[1:Ns])
    C_T = fftshift(xcorr(T[1:Ns] - mT, T[1:Ns] - mT))[1:100]

    rho_T = C_T/C_T[1]

    plot( rho_T, "-", label="N=$Ns")
end
legend(); xlabel("iteration"), ylabel(L"\rho_T")
close(gcf())

At long times, the autocorrelation decays exponentially,

$$
\rho_{xx}(t) \sim e^{-|t|/\tau}
$$

The __exponential autocorrelation time__ for an observable is defined as

$$
\tau_{\text{exp},x} = \limsup_{t\rightarrow\infty} \frac{t}{-\log |\rho_{xx}(t)|}
$$

If we want to be sure that all correlations have decayed, we need to take the _longest_ exponental autocorrelation time, so for the whole system,

$$
\tau_\text{exp} = \sup_{x} \tau_{\text{exp},x}
$$

(If we are unlucky, this might not be finite!)

* $\tau_\text{exp}$ is used to measure _relaxation time_
* Advisable to discard the first $10 \tau$ amount simulation before we measure anything!

### But what about the error on sample mean? 

* Measurements that comprise the mean, e.g. x(i) and x(i+1) etc., are _not_ independent
* The autocorrelation is precisely the thing that measures their covariance

$$
\begin{eqnarray}
\text{Var}(\bar x) &=& \frac{1}{N^2} \sum_{i,j=1}^N C_{xx}(i-j)\\
&=& \frac{1}{N^2} \sum_{t=-(N-1)}^{N-1} (N - |t|) C_{xx}(t)\\
&=& \frac1N \sum_{t=-(N-1)}^{N-1} \left(1-\frac{|t|}{N}\right) C_{xx}(t)\\
&\approx& \frac1N \sum_{t=-\infty}^{\infty} C_{xx}(t) \qquad \text{for } n \gg \tau_\text{exp}\\
\end{eqnarray}
$$

Define the __integrated autocorrelation time__, 

$$
\begin{eqnarray}
2 \tau_{\text{int},x} &=& \frac{1}{C_{xx}(0)} \sum_{t=-\infty}^{\infty} C_{xx}(t)\\
 &=& \sum_{t=-\infty}^{\infty} \rho_{xx}(t)\\
 &=& 1+\sum_{t=1}^{\infty} \rho_{xx}(t)
\end{eqnarray} 
$$

So

$$
\text{Var}(\bar x) \approx \frac1N (2\tau_{\text{int},x}) C_{xx}(0)
$$

The number of _effectively independent samples_ is a factor $2\tau_{\text{int},x}$ less than $N$,

$$
\text{Std}(\bar x) = \frac{\text{Std}(x)}{\sqrt{N/2\tau_{\text{int},x}}}
$$

The autocorrelation times $\tau_\text{exp}$ and $\tau_\text{int}$ are often assumed to be of the same order of magnitude. This is certainly not true near critical points associated with phase transitions. 

#### Caveats

* We need the simulation to be long enough for the $\tau_\text{int}$ formula to be valid
* We can only estimate the autocorrelation, and hence $\tau_\text{int}$
* In computing the integrated autocorrelation, we should _stop_ adding up the values at some point, otherwise the result will start diverging

#### Estimating $\tau_\text{int}$

In [ ]:
figure()

for Ns in [100, 1000, 10000, 100000]
    Tm = mean(T[1:Ns])
    C_T = fftshift(xcorr(T[1:Ns] - Tm, T[1:Ns] - Tm))[1:100]

    rho_T = C_T/C_T[1]

    twotau = 1+cumsum(rho_T)
    plot(twotau, "-", label="N=$Ns")
end
legend(loc="center left", bbox_to_anchor=(1,0.5)); xlabel("iteration"), ylabel(L"2\tau_{\mathrm{int},T}")
close(gcf())

In [ ]:
start=1
Ns = [100, 1000, 10000, 100000]
Tm = []
Tmerr = []
for n in Ns
    push!(Tm, mean(T[start:n]))
    
    C_T = fftshift(xcorr(T[start:n] - Tm[end], T[start:n] - Tm[end]))[1:10]
    rho_T = C_T/C_T[1]
    twotau = 1+sum(rho_T)
    
    push!(Tmerr, std(T[start:n])/sqrt((n-start+1)/twotau))
end

figure()
errorbar(log10(Ns), Tm, Tmerr, fmt="o-")
axis([1,6,360,420]); xlabel(L"\log_{10}(N)"); ylabel("kinetic T")
close(gcf())
    


## More complicated observables

Radius of gyration (of the oxygens):

$$
R_g = \left[\frac{1}{6} \sum_{i=1}^6 \left|{\bf r }_{O_i}-\bar{\bf r }\right|^2\right]^\frac{1}{2}
$$



In [ ]:
figure()
plot(1:100:100000, Rg[1:100:100000], "-", label="Radius of gyration")
plot([1,N], repeat([mean(Rg)], inner=[2]), "k--", label="mean")
plot([1,N], repeat([mean(Rg)+std(Rg)/sqrt(N/twotau)], inner=[2]), "r--")
plot([1,N], repeat([mean(Rg)-std(Rg)/sqrt(N/twotau)], inner=[2]), "r--")
xlabel("iteration"); ylabel("Rg: Radius of gyration  / A")
close(gcf())

In [ ]:
C = fftshift(xcorr(Rg - mean(Rg), Rg - mean(Rg)))[1:length(Rg)÷2]
rho = C/C[1]

figure()
plot(1:10:length(rho), rho[1:10:end])
xlabel("iteration"); ylabel(L"\rho_{Rg}")
close(gcf())

In [ ]:
twotau = 2+sum(rho[1:10000])

#### Orientation of Water1

$$
w1z = z_{H_1}-z_{O_1}
$$

In [ ]:
figure()
plot(1:100:100000, w1z[1:100:100000], "-")
plot([1,N], repeat([mean(w1z)], inner=[2]), "k--")
plot([1,N], repeat([mean(w1z)+std(w1z)/sqrt(N/twotau)], inner=[2]), "r--")
plot([1,N], repeat([mean(w1z)-std(w1z)/sqrt(N/twotau)], inner=[2]), "r--")
xlabel("iteration"); ylabel("Water 1 OH(z)")
close(gcf())

In [ ]:
C = fftshift(xcorr(w1z - mean(w1z), w1z - mean(w1z)))[1:length(w1z)÷2]
rho = C/C[1]

figure()
plot(1:10:length(rho), rho[1:10:end])
xlabel("iteration"); ylabel(L"\rho_{w1z}")
close(gcf())

In [ ]:
twotau = 2+sum(rho[1:10000])

## Radius of gyration as a function of temperature

In [ ]:
## Takes a long time§§ to run 
##

Ts = [100.0,200.0,300.0]
Rgs = []
Rges = []
for T in Ts
    println("T = $T K")
    myRg,myw1z = water_hexamer_dynamics(;T=T, Nsteps=500000, Nsubsamp=100)
    Rgm,Rge = mean_err_corr(myRg[1000:end]; acorr_limit=100)
    push!(Rgs, Rgm)
    push!(Rges, Rge)
end

In [ ]:
figure()
errorbar(Ts, Rgs ,Rges, fmt=".-")
axis([minimum(Ts)-50, maximum(Ts)+50, 2.4, 3.1]); xlabel("Temperature / K"); ylabel("Rg")
close(gcf())

In [ ]:
# very short run !!

sTs = [100.0,200.0,300.0]
sRgs = []
sRges = []
for T in Ts
    println("T = $T K")
    myRg,myw1z = water_hexamer_dynamics(;T=T, Nsteps=10000, Nsubsamp=10)
    Rgm,Rge = mean_err_corr(myRg[200:end]; acorr_limit=50)
    push!(sRgs, Rgm)
    push!(sRges, Rge)
end
figure()
errorbar(sTs, sRgs ,sRges, fmt="o-")
axis([minimum(sTs)-50, maximum(sTs)+50, 2.4, 3.3]); xlabel("Temperature / K"); ylabel("Rg")
close(gcf())


## Complicated observables: errors in histograms

#### O-O distance distribution in the water dimer

In [ ]:
rOO_long = water_dimer_dynamics(Temp=100.0, Nsteps=100000, Nsave=0)
;

In [ ]:
b0,p0 = hist(rOO_long, 2.4:0.05:3.2)
figure()
plot(midpoints(b0), p0/length(rOO_long)/(b0[2]-b0[1]), "ko--", label="100000 steps")
legend(); xlabel("rOO / A"), ylabel("P(rOO)")
close(gcf())

In [ ]:
rOO_short = water_dimer_dynamics(Temp=100.0, Nsteps=1000, Nsave=0)
b,p = hist(rOO_short, 2.4:0.1:3.2)
b0,p0 = hist(rOO_long, 2.4:0.1:3.2)
;

In [ ]:
figure()
plot(midpoints(b), p/length(rOO_short)/(b[2]-b[1]), "o-", label="1000 steps")
plot(midpoints(b0), p0/length(rOO_long)/(b0[2]-b0[1]), "ko--", label="100000 steps")
legend(); xlabel("rOO / A"), ylabel("P(rOO)")
close(gcf())

In [ ]:
blocks = 100
blocksize = length(rOO_short)÷blocks
r = 2.4:0.1:3.2
p = zeros(length(midpoints(r)), blocks)
for i=1:blocks
    b,p[:,i] = hist(rOO_short[(i-1)*blocksize+1:i*blocksize ], r)
    p[:,i] /= (blocksize)*(b[2]-b[1])
end

figure()
for i=1:20
    plot(midpoints(b), p[:,i], "-")
end
close(gcf())

In [ ]:
p

In [ ]:
pp = zeros(size(p,1)); ppe = zeros(pp)
for i=1:length(pp)
    pp[i],ppe[i] = mean_err_corr(vec(p[i,:]), acorr_limit=10)
end
[pp ppe]

In [ ]:
ppe[find(isnan(ppe))] = 0
[pp ppe]

In [ ]:
figure()
errorbar(midpoints(r), pp, ppe, fmt="o-")
plot(midpoints(b0), p0/length(rOO_long)/(b0[2]-b0[1]), "ko--")
close(gcf())